<a href="https://colab.research.google.com/github/Loga19818eeanvesh/CNN_Models/blob/main/2019_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install tensorflow --upgrade
!pip install keras --upgrade

     |████████████████████████████████| 497.5 MB 17 kB/s 
     |████████████████████████████████| 5.8 MB 39.0 MB/s 
     |████████████████████████████████| 462 kB 46.7 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [1]:
import tensorflow as tf 
print(tf.__version__)

2.8.0


In [2]:
from tensorflow.keras import Model
from tensorflow.keras.layers import DepthwiseConv2D,Conv2D,BatchNormalization,GlobalAveragePooling2D,Flatten,Add,InputLayer,ReLU,ZeroPadding2D
from tensorflow.keras.activations import relu,softmax
from tensorflow.nn import relu6

In [3]:
class DottleneckDepthSeparableConvolution(Model):
  def __init__(self,t,c,f,stride=1):
    super().__init__()
    self.expansion = Conv2D(filters=(t*c),kernel_size=1)
    self.stride = stride
    if stride==2:
      self.zero_pad = ZeroPadding2D(padding=((1, 0), (1, 0)))
      self.depthwise_conv = DepthwiseConv2D(kernel_size=3,padding='valid',strides=2)
    else:
      self.depthwise_conv = DepthwiseConv2D(kernel_size=3,padding='same')
    self.projection = Conv2D(filters=f,kernel_size=1)
    self.relu_activation =  relu6
    self.add = Add()

  def call(self,inputs):
    x = self.expansion(inputs)
    x = self.relu_activation(x)
    if self.stride==2:
      x = self.zero_pad(x)
    x = self.depthwise_conv(x)
    x = self.relu_activation(x)
    x = self.projection(x)
    output = x
    if(x.shape==inputs.shape):
      output = self.add([x,inputs])
      
    return output

In [ ]:
'''
class DottleneckDepthSeparableConvolutionStride2Block(Model):
  def __init__(self,t,c,f):
    super().__init__()
    self.channels = t*c
    self.expansion = Conv2D(filters=(t*c),kernel_size=1)
    self.depthwise_conv = depthwise_conv2d
    self.projection = Conv2D(filters=f,kernel_size=1)
    self.relu_activation = ReLU()
    f_init = tf.random_normal_initializer()
    self.filter_init = tf.Variable(initial_value=f_init(shape=(3,3,(t*c),1) , dtype="float32") , trainable=True)
    self.bias = tf.nn.bias_add
    self.b = tf.Variable(tf.zeros(shape = [(t*c),]), dtype="float32")

  def call(self,inputs):
    x = self.expansion(inputs)
    x = self.relu_activation(x)
    x = self.depthwise_conv(x,filter=self.filter_init,strides = [1, 2, 2, 1],padding=[[0, 0], [1,0],[1,0], [0, 0]])
    x = self.bias(x, self.b)
    x = self.relu_activation(x)
    x = self.projection(x)

    return x
    '''

In [4]:
class MobileNetBlock(Model):
  def __init__(self,t,c,f,n,stride):
    super().__init__()
    self.stride2 = DottleneckDepthSeparableConvolution(t=t,c=c,f=f,stride=stride)
    self.nl = n-1
    self.layers1 = {}
    for i in range(n-1):
      self.layers1["stride1"+str(i)] = DottleneckDepthSeparableConvolution(t=t,c=c,f=f)

  def call(self,inputs):
    x = self.stride2(inputs)
    for i in range(self.nl):
      x = self.layers1["stride1"+str(i)](x)
    return x

In [5]:
class MobileNetV2(Model):
  def __init__(self,input_shape,noof_classes):
    super().__init__()
    self.input1 = InputLayer(input_shape=input_shape)
    self.zero_pad = ZeroPadding2D(padding=((1, 0), (1, 0)))
    self.relu_activation =  relu6
    self.conv1 = Conv2D(filters=32,kernel_size=3,strides=2,padding='valid')
    self.bottleneck1 = DottleneckDepthSeparableConvolution(t=1,c=32,f=16)
    self.mobile_net_block1 = MobileNetBlock(t=6,c=16,f=24,n=2,stride=2)
    self.mobile_net_block2 = MobileNetBlock(t=6,c=24,f=32,n=3,stride=2)
    self.mobile_net_block3 = MobileNetBlock(t=6,c=32,f=64,n=4,stride=2)
    self.mobile_net_block4 = MobileNetBlock(t=6,c=64,f=96,n=3,stride=1)
    self.mobile_net_block5 = MobileNetBlock(t=6,c=96,f=160,n=3,stride=2)
    self.bottleneck5 = DottleneckDepthSeparableConvolution(t=6,c=160,f=320)
    self.conv2 = Conv2D(kernel_size=1,filters=1280)
    self.global_avg_pool = GlobalAveragePooling2D(keepdims=True)
    self.conv3 = Conv2D(kernel_size=1,filters=noof_classes)
    self.flattten = Flatten()
    self.classifier = softmax

  def call(self,inputs):
    x = self.input1(inputs)
    x = self.zero_pad(x)
    x = self.conv1(x)
    x = self.relu_activation(x)
    x = self.bottleneck1(x)
    x = self.mobile_net_block1(x)
    x = self.mobile_net_block2(x)
    x = self.mobile_net_block3(x)
    x = self.mobile_net_block4(x)
    x = self.mobile_net_block5(x)
    x = self.bottleneck5(x)
    x = self.conv2(x)
    x = self.global_avg_pool(x)
    x = self.conv3(x)
    x = self.flattten(x)
    x = self.classifier(x)

    return x
    

In [6]:
MobileNetV2_model = MobileNetV2(input_shape=(None,224,224,3),noof_classes=1000)

In [9]:
MobileNetV2_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                       metrics=tf.keras.metrics.SparseCategoricalAccuracy()
                        )

In [7]:
MobileNetV2_model.build(input_shape=(None,224,224,3)) 

In [8]:
MobileNetV2_model.summary()

Model: "mobile_net_v2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 224, 224,   0         
                             3)]                                 
                                                                 
 zero_padding2d (ZeroPadding  multiple                 0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             multiple                  896       
                                                                 
 dottleneck_depth_separable_  multiple                 1904      
 convolution (DottleneckDept                                     
 hSeparableConvolution)                                          
                                                                 
 mobile_net_block (MobileNet  multiple               